In [11]:
import pandas as pd
data = pd.read_csv('../staging/data_set/master_dataset.csv')

In [4]:
from sklearn.feature_selection import SelectKBest, f_classif

In [5]:
data['arr_change'] = data['future_arr'] - data['current_arr']
data['perc_change'] = (data['future_arr'] - data['current_arr']) / data['current_arr']
data['churn_flag'] = (data['future_arr'] < data['current_arr'] * 0.8).astype(int)

In [6]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
le_industry = LabelEncoder()
le_employee = LabelEncoder()

# Fit and transform the columns
data['crm_industry_current'] = le_industry.fit_transform(data['crm_industry_current'])
data['crm_employee_range'] = le_employee.fit_transform(data['crm_employee_range'])

# Print the mapping of values to integers for each column
print("Mapping for 'crm_industry_current':")
print(dict(zip(le_industry.classes_, range(len(le_industry.classes_)))))

print("\nMapping for 'crm_employee_range':")
print(dict(zip(le_employee.classes_, range(len(le_employee.classes_)))))

Mapping for 'crm_industry_current':
{'0': 0, 'Consumer Services': 1, 'Corporate Services': 2, 'Education': 3, 'Energy & Utilities': 4, 'Financial Services': 5, 'Government': 6, 'Healthcare': 7, 'Manufacturing & Resources': 8, 'Media & Entertainment': 9, 'Non-Profits & Associations': 10, 'Retail, Wholesale & Distribution': 11, 'Technology': 12, 'Telecommunications': 13, 'Transportation': 14, 'Travel & Hospitality': 15}

Mapping for 'crm_employee_range':
{'01-Sep': 0, '100 - 249': 1, '1000 - 4999': 2, '250 - 499': 3, '50 - 99': 4, '500 - 999': 5, '5000+': 6, 'Oct-49': 7}


In [7]:
# Remove the 'id' column from X
X1 = data.drop(columns=['id' , 'score_date' , 'perc_change'], errors='ignore')  # Use 'errors="ignore"' to avoid errors if 'id' is not present



In [8]:
X = X1.iloc[:, :-1]  
y = X1.iloc[:, -1] 
# Check columns in X
print("Columns in X:")
print(X.columns)

# Check the name of the target column (y)
print("\nTarget column (y):")
print(y.name)

Columns in X:
Index(['segment_smb', 'segment_non_smb', 'segment_commercial',
       'segment_enterprise', 'segment_midmarket', 'region_emea', 'region_apac',
       'region_latam', 'region_amer', 'customer_age_quarters',
       'sales_assisted', 'legacy_flag', 'product_counts',
       'billing_dunning_not_ok', 'crm_industry_current', 'crm_employee_range',
       'crm_success_owner_change', 'current_arr', 'future_arr', 'arr_change',
       'fx_impact', 'seat_change_arr', 'product_change_arr',
       'discount_arr_usd', 'team_plus', 'pro_plus', 'ent_plus',
       'discount_arr_usd_percentage', 'product_counts_percentage',
       'total_pool_max_agents', 'max_seats', 'agent_utilization',
       'is_provisioned_any_channel_M_before', 'max_seats_M_before',
       'agent_utilization_increase', 'agent_utilization_decrease',
       'seat_utilization', 'seat_utilization_M_before',
       'seat_utilization_increase', 'seat_utilization_decrease', 'csat_score',
       'csat_response_rate', 'nps_sco

In [12]:
import pandas as pd

# Perform feature selection
selector = SelectKBest(score_func=f_classif, k=10)
X_new = selector.fit_transform(X, y)

# Get the selected feature indices
selected_features = selector.get_support(indices=True)

# Get the names of the selected features (if X is a DataFrame)
if isinstance(X, pd.DataFrame):
    feature_names = X.columns[selected_features]
    print("Selected Features:")
    print(feature_names)
else:
    print("Selected feature indices:")
    print(selected_features)

# Output the transformed feature matrix
print("Transformed Feature Matrix:")
print(X_new)

Selected Features:
Index(['sales_assisted', 'current_arr', 'arr_change', 'seat_change_arr',
       'product_change_arr', 'pro_plus', 'max_seats',
       'is_provisioned_any_channel_M_before', 'max_seats_M_before',
       'seat_utilization_decrease'],
      dtype='object')
Transformed Feature Matrix:
[[1.00000000e+00 2.97600000e+03 0.00000000e+00 ... 2.00000000e+00
  2.00000000e+00 0.00000000e+00]
 [1.00000000e+00 2.07247200e+04 7.50120000e+02 ... 9.00000000e+00
  9.00000000e+00 0.00000000e+00]
 [0.00000000e+00 6.05760000e+02 2.12040000e+02 ... 1.00000000e+00
  1.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 2.28000000e+02 0.00000000e+00 ... 1.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.28000000e+02 0.00000000e+00 ... 8.78967286e+00
  1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.08000000e+03 0.00000000e+00 ... 8.00000000e+00
  1.00000000e+01 0.00000000e+00]]


c:\Users\FahmidaSanabilSyedAb\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [11 29] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\FahmidaSanabilSyedAb\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [10]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X, y)